# EncypherAI PDF Generation Demo

This notebook demonstrates how to use the EncypherPDF class to create PDFs with embedded metadata and verify them.

## Setup

First, let's import the necessary modules and generate a key pair for signing.

In [ ]:
import os
from datetime import datetime, timezone
from cryptography.hazmat.primitives.asymmetric import ed25519
from IPython.display import IFrame, display, Markdown

from encypher.pdf_generator import EncypherPDF
from encypher.core.unicode_metadata import UnicodeMetadata

# Create a directory for our output files
os.makedirs("output", exist_ok=True)

In [ ]:
# Generate a key pair for signing
private_key = ed25519.Ed25519PrivateKey.generate()
public_key = private_key.public_key()

# Define a signer ID and timestamp
signer_id = "example-signer"
timestamp = int(datetime.now(timezone.utc).timestamp())
timestamp_str = datetime.fromtimestamp(timestamp, timezone.utc).isoformat()

print(f"Generated key pair with signer ID: {signer_id}")
print(f"Current timestamp: {timestamp} ({timestamp_str})")

## Basic PDF Generation

Let's create a simple PDF with embedded metadata.

In [ ]:
# Define the text content and output path
text = "Hello from EncypherAI! This is a simple PDF with embedded metadata."
output_path = "output/basic_example.pdf"

# Generate the PDF
pdf_path = EncypherPDF.from_text(
    text=text,
    output_path=output_path,
    private_key=private_key,
    signer_id=signer_id,
    timestamp=timestamp,
)

print(f"PDF created at: {pdf_path}")

Let's display the PDF we just created:

In [ ]:
# Display the PDF in the notebook
display(IFrame(output_path, width=700, height=400))

## Verifying the PDF

Now let's extract the text from the PDF and verify the embedded metadata.

In [ ]:
# Extract text from the PDF
extracted_text = EncypherPDF.extract_text(output_path)
print("Extracted text:")
print(extracted_text)

In [ ]:
# Define a public key resolver function
def public_key_resolver(sid):
    # In a real application, you would look up the public key based on the signer ID
    return public_key if sid == signer_id else None

# Verify the PDF
is_valid, extracted_signer_id, payload = EncypherPDF.verify_pdf(
    output_path,
    public_key_resolver=public_key_resolver,
)

print(f"Verification result: {'Valid' if is_valid else 'Invalid'}")
print(f"Signer ID: {extracted_signer_id}")
print(f"Payload: {payload}")

## Advanced Features

### Custom Fonts

EncypherPDF supports custom TrueType fonts for better Unicode support. Let's create a PDF with a custom font if available.

In [ ]:
# Check if a font file exists (you can replace this with your own font path)
font_path = "path/to/your/font.ttf"  # Replace with actual path if available
font_exists = os.path.exists(font_path)

if font_exists:
    # Text with Unicode characters
    unicode_text = "Hello with custom font! 你好，世界！こんにちは！안녕하세요!"
    unicode_output_path = "output/unicode_example.pdf"
    
    # Generate PDF with custom font
    unicode_pdf_path = EncypherPDF.from_text(
        text=unicode_text,
        output_path=unicode_output_path,
        private_key=private_key,
        signer_id=signer_id,
        timestamp=timestamp,
        font_path=font_path,
        font_name="CustomFont",
        font_size=14,
    )
    
    print(f"Unicode PDF created at: {unicode_pdf_path}")
    display(IFrame(unicode_output_path, width=700, height=400))
else:
    print("Custom font file not found. Using default font instead.")
    
    # Text with Unicode characters (may not display correctly with default font)
    unicode_text = "Hello with default font! Some Unicode characters may not display correctly: 你好，世界！"
    unicode_output_path = "output/unicode_default_font.pdf"
    
    # Generate PDF with default font
    unicode_pdf_path = EncypherPDF.from_text(
        text=unicode_text,
        output_path=unicode_output_path,
        private_key=private_key,
        signer_id=signer_id,
        timestamp=timestamp,
    )
    
    print(f"Unicode PDF with default font created at: {unicode_pdf_path}")
    display(IFrame(unicode_output_path, width=700, height=400))

### Different Metadata Formats

EncypherPDF supports different metadata formats. Let's create PDFs with different formats.

In [ ]:
# Define the text content
format_text = "This PDF uses a different metadata format."

# List of formats to demonstrate
formats = ["basic", "manifest", "cbor_manifest", "c2pa_v2_2"]

for fmt in formats:
    output_path = f"output/{fmt}_format.pdf"
    
    # Generate PDF with the specified format
    pdf_path = EncypherPDF.from_text(
        text=format_text,
        output_path=output_path,
        private_key=private_key,
        signer_id=signer_id,
        timestamp=timestamp,
        metadata_format=fmt,
    )
    
    print(f"PDF with {fmt} format created at: {pdf_path}")
    
    # Verify the PDF
    is_valid, extracted_signer_id, payload = EncypherPDF.verify_pdf(
        output_path,
        public_key_resolver=public_key_resolver,
    )
    
    print(f"Verification result: {'Valid' if is_valid else 'Invalid'}")
    print(f"Payload type: {type(payload).__name__}")
    print("---")

### Custom Metadata

Let's create a PDF with custom metadata.

In [ ]:
# First, create metadata-rich text with custom metadata
custom_metadata = {
    "title": "Example PDF",
    "author": "EncypherAI",
    "description": "A PDF with custom metadata",
    "keywords": ["EncypherAI", "PDF", "metadata", "digital signatures"],
    "version": "1.0.0"
}

# Embed metadata in text
text_with_metadata = UnicodeMetadata.embed_metadata(
    text="This PDF contains custom metadata.",
    private_key=private_key,
    signer_id=signer_id,
    timestamp=timestamp,
    custom_metadata=custom_metadata
)

# Create PDF with the metadata-rich text
custom_metadata_path = "output/custom_metadata.pdf"
pdf_path = EncypherPDF.from_text(
    text=text_with_metadata,
    output_path=custom_metadata_path,
    private_key=private_key,
    signer_id=signer_id,
    timestamp=timestamp,
)

print(f"PDF with custom metadata created at: {pdf_path}")
display(IFrame(custom_metadata_path, width=700, height=400))

In [ ]:
# Verify the PDF with custom metadata
is_valid, extracted_signer_id, payload = EncypherPDF.verify_pdf(
    custom_metadata_path,
    public_key_resolver=public_key_resolver,
)

print(f"Verification result: {'Valid' if is_valid else 'Invalid'}")
print(f"Signer ID: {extracted_signer_id}")

# Display the custom metadata
if is_valid and payload:
    print("\nCustom Metadata:")
    custom_metadata = payload.custom_metadata if hasattr(payload, 'custom_metadata') else {}
    for key, value in custom_metadata.items():
        print(f"  {key}: {value}")

## Error Handling

EncypherPDF includes robust error handling. Let's demonstrate some error scenarios.

In [ ]:
# Try to verify a PDF with an incorrect public key
def wrong_key_resolver(sid):
    # This will always return None, simulating a missing or incorrect key
    return None

try:
    is_valid, extracted_signer_id, payload = EncypherPDF.verify_pdf(
        output_path,
        public_key_resolver=wrong_key_resolver,
    )
    print(f"Verification result: {'Valid' if is_valid else 'Invalid'}")
    print(f"Signer ID: {extracted_signer_id}")
except Exception as e:
    print(f"Error during verification: {e}")

In [ ]:
# Try to create a PDF with a non-existent font
try:
    pdf_path = EncypherPDF.from_text(
        text="This will fail due to a non-existent font.",
        output_path="output/font_error.pdf",
        private_key=private_key,
        signer_id=signer_id,
        timestamp=timestamp,
        font_path="non_existent_font.ttf",
    )
except Exception as e:
    print(f"Error creating PDF with non-existent font: {e}")

## Conclusion

This notebook has demonstrated the key features of the EncypherPDF class:

1. Creating PDFs with embedded metadata
2. Verifying PDFs and extracting metadata
3. Using custom fonts for better Unicode support
4. Working with different metadata formats
5. Adding custom metadata
6. Handling errors gracefully

The EncypherPDF class provides a simple and secure way to create and verify PDFs with embedded metadata, making it easy to integrate into your workflow.